# Manifold GP Semi-Supervised Learning via Precision Matrix on 1D Manifold

## Preamble

This notebook provides an example of how to perform Gaussian Process Regression on a 1D manifold. In this example we consider a supervised learning scenario, namely the number of labeled data points is equivalent to the number of the sampled points from the underlying manifold.

In [1]:
import torch
import gpytorch
import numpy as np
import os
import scipy.spatial as ss

from manifold_gp.kernels.riemann_matern_kernel import RiemannMaternKernel
from manifold_gp.models.riemann_gp import RiemannGP
from gpytorch.priors import NormalPrior, GammaPrior

## Dataset Preprocessing

### Load & Settings

In [2]:
dataset = 'protein' # ['protein','elevators']
cut = 10000

if dataset == 'protein':
    data = np.loadtxt('protein.csv', delimiter=",")[:cut]
    sampled_x, sampled_y = data[:, 1:], data[:, 0]
elif dataset == 'elevators':
    data = np.array(loadmat('elevators.mat')['data'])
    sampled_x, sampled_y = data[:, :-1], data[:, -1]
    
    
if not os.path.isfile(f'../outputs/{dataset}.mat'):
    print(f'Downloading \'{dataset}\' UCI dataset...')
    urllib.request.urlretrieve(datasets[dataset], f'../outputs/{dataset}.mat')

# remove coincident points
sampled_x, id_unique = np.unique(sampled_x, axis=0, return_index=True)
sampled_y = sampled_y[id_unique]

# cut between 0.01 and 0.99 quantile of distances
kd_tree = ss.KDTree(sampled_x)
v = kd_tree.query(sampled_x, k=2)[0][:, 1]
idx = np.argsort(v)
percentile_start = int(np.round(idx.shape[0]*0.10))
percentile_end = int(np.round(idx.shape[0]*0.90))
sampled_x = sampled_x[idx[percentile_start:percentile_end], :]
sampled_y = sampled_y[idx[percentile_start:percentile_end]]
m = sampled_x.shape[0]

normalize_features = True
normalize_labels = True

### Trainset & Testset

In [3]:
split = int(0.8 * m)

train_x, train_y = sampled_x[:split], sampled_y[:split]
test_x, test_y = sampled_x[split:], sampled_y[split:]

train_idx = torch.arange(0, split)
sampled_x = torch.from_numpy(sampled_x).float()
train_x = torch.from_numpy(train_x).float()
train_y = torch.from_numpy(train_y).float()
test_x = torch.from_numpy(test_x).float()
test_y = torch.from_numpy(test_y).float()

if normalize_features:
    mu_x, std_x = sampled_x.mean(dim=-2, keepdim=True), sampled_x.std(dim=-2, keepdim=True) + 1e-6
    sampled_x.sub_(mu_x).div_(std_x)
    train_x.sub_(mu_x).div_(std_x)
    test_x.sub_(mu_x).div_(std_x)
    
if normalize_labels:
    mu_y, std_y = train_y.mean(), train_y.std()
    train_y.sub_(mu_y).div_(std_y)
    test_y.sub_(mu_y).div_(std_y)

### Hyperparameters Priors

In [4]:
import scipy.spatial as ss
neighbors = 10
kd_tree = ss.KDTree(sampled_x)
v = np.sort(kd_tree.query(sampled_x, k=neighbors+1)[0][:, 1:].ravel())
percentile_99 = int(np.round(v.shape[0]*0.99))
gamma_rate = 100.0/np.std(v)
gamma_concentration = gamma_rate * v[percentile_99] + 1

### Move Data to Device

In [5]:
sampled_x = sampled_x.contiguous()
# train_idx = train_idx.contiguous()
train_x, train_y = train_x.contiguous(), train_y.contiguous()
test_x, test_y = test_x.contiguous(), test_y.contiguous()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
sampled_x = sampled_x.to(device)
# train_idx = train_idx.to(device)
train_x, train_y = train_x.to(device), train_y.to(device)
test_x, test_y = test_x.to(device), test_y.to(device)

## Model

In [6]:
%%capture
likelihood = gpytorch.likelihoods.GaussianLikelihood(
    noise_constraint=gpytorch.constraints.GreaterThan(1e-8),
    noise_prior=None  # NormalPrior(torch.tensor([0.0]).to(device),  torch.tensor([1/9]).sqrt().to(device))
)

kernel = gpytorch.kernels.ScaleKernel(
    RiemannMaternKernel(
        nu=1,
        nodes=sampled_x,
        neighbors=10,
        operator="randomwalk",
        modes=50,
        ball_scale=1.0,
        support_kernel=gpytorch.kernels.RBFKernel(),
        epsilon_prior=GammaPrior(gamma_concentration, gamma_rate),
        lengthscale_prior=None, # InverseGammaPrior(igamma_concentration, igamma_rate)
    ),
    outputscale_prior=None  # NormalPrior(torch.tensor([1.0]).to(device),  torch.tensor([1/9]).sqrt().to(device))
)

model = RiemannGP(train_x, train_y, likelihood, kernel, train_idx).to(device)

## Train

In [7]:
%%capture
hypers = {
    'likelihood.noise_covar.noise': 1e-2, # 0.037,
    'covar_module.base_kernel.epsilon': 0.5, # 0.029,
    'covar_module.base_kernel.lengthscale': 0.5, # 5.130,
    'covar_module.outputscale': 1.0, # 0.0656,
    'covar_module.base_kernel.support_kernel.lengthscale': 1.0,
}
model.initialize(**hypers)

In [9]:
model.manifold_informed_train(lr=1e-2, iter=100, norm_step_size=100, verbose=True)

Iter: 0, LR: 0.010, Loss: 4264.340, NoiseVar: 0.000, SignalVar: 1379.365, Lengthscale: 0.514, Epsilon: 0.249
Iter: 1, LR: 0.010, Loss: 4172.298, NoiseVar: 0.000, SignalVar: 1379.375, Lengthscale: 0.518, Epsilon: 0.251
Iter: 2, LR: 0.010, Loss: 4102.246, NoiseVar: 0.000, SignalVar: 1379.385, Lengthscale: 0.522, Epsilon: 0.253
Iter: 3, LR: 0.010, Loss: 3990.775, NoiseVar: 0.000, SignalVar: 1379.395, Lengthscale: 0.526, Epsilon: 0.256
Iter: 4, LR: 0.010, Loss: 3924.222, NoiseVar: 0.000, SignalVar: 1379.405, Lengthscale: 0.530, Epsilon: 0.258
Iter: 5, LR: 0.010, Loss: 3839.879, NoiseVar: 0.000, SignalVar: 1379.415, Lengthscale: 0.534, Epsilon: 0.260
Iter: 6, LR: 0.010, Loss: 3772.016, NoiseVar: 0.000, SignalVar: 1379.425, Lengthscale: 0.538, Epsilon: 0.263
Iter: 7, LR: 0.010, Loss: 3663.857, NoiseVar: 0.000, SignalVar: 1379.435, Lengthscale: 0.542, Epsilon: 0.265
Iter: 8, LR: 0.010, Loss: 3573.670, NoiseVar: 0.000, SignalVar: 1379.445, Lengthscale: 0.546, Epsilon: 0.267
Iter: 9, LR: 0.010,

Iter: 75, LR: 0.010, Loss: 2261.324, NoiseVar: 0.000, SignalVar: 1379.776, Lengthscale: 0.725, Epsilon: 0.307
Iter: 76, LR: 0.010, Loss: 2230.555, NoiseVar: 0.000, SignalVar: 1379.777, Lengthscale: 0.726, Epsilon: 0.306
Iter: 77, LR: 0.010, Loss: 2236.580, NoiseVar: 0.000, SignalVar: 1379.779, Lengthscale: 0.728, Epsilon: 0.305
Iter: 78, LR: 0.010, Loss: 2226.096, NoiseVar: 0.000, SignalVar: 1379.780, Lengthscale: 0.729, Epsilon: 0.304
Iter: 79, LR: 0.010, Loss: 2262.368, NoiseVar: 0.000, SignalVar: 1379.781, Lengthscale: 0.730, Epsilon: 0.303
Iter: 80, LR: 0.010, Loss: 2237.590, NoiseVar: 0.000, SignalVar: 1379.782, Lengthscale: 0.731, Epsilon: 0.302
Iter: 81, LR: 0.010, Loss: 2220.245, NoiseVar: 0.000, SignalVar: 1379.783, Lengthscale: 0.733, Epsilon: 0.301
Iter: 82, LR: 0.010, Loss: 2239.071, NoiseVar: 0.000, SignalVar: 1379.784, Lengthscale: 0.734, Epsilon: 0.301
Iter: 83, LR: 0.010, Loss: 2238.035, NoiseVar: 0.000, SignalVar: 1379.785, Lengthscale: 0.735, Epsilon: 0.300
Iter: 84, 

In [10]:
model.manifold_informed_train(lr=1e-1, iter=100, norm_step_size=100, verbose=True)

Iter: 0, LR: 0.100, Loss: 2227.841, NoiseVar: 0.000, SignalVar: 1400.488, Lengthscale: 0.754, Epsilon: 0.294
Iter: 1, LR: 0.100, Loss: 2160.583, NoiseVar: 0.000, SignalVar: 1400.388, Lengthscale: 0.808, Epsilon: 0.269
Iter: 2, LR: 0.100, Loss: 2197.710, NoiseVar: 0.000, SignalVar: 1400.306, Lengthscale: 0.859, Epsilon: 0.287
Iter: 3, LR: 0.100, Loss: 2203.391, NoiseVar: 0.000, SignalVar: 1400.228, Lengthscale: 0.880, Epsilon: 0.277
Iter: 4, LR: 0.100, Loss: 2162.804, NoiseVar: 0.000, SignalVar: 1400.142, Lengthscale: 0.883, Epsilon: 0.263
Iter: 5, LR: 0.100, Loss: 2160.707, NoiseVar: 0.000, SignalVar: 1400.052, Lengthscale: 0.880, Epsilon: 0.252
Iter: 6, LR: 0.100, Loss: 2179.972, NoiseVar: 0.000, SignalVar: 1399.964, Lengthscale: 0.877, Epsilon: 0.250
Iter: 7, LR: 0.100, Loss: 2155.956, NoiseVar: 0.000, SignalVar: 1399.878, Lengthscale: 0.876, Epsilon: 0.255
Iter: 8, LR: 0.100, Loss: 2150.128, NoiseVar: 0.000, SignalVar: 1399.792, Lengthscale: 0.876, Epsilon: 0.263
Iter: 9, LR: 0.100,

Iter: 75, LR: 0.100, Loss: 2153.367, NoiseVar: 0.000, SignalVar: 1393.466, Lengthscale: 0.837, Epsilon: 0.267
Iter: 76, LR: 0.100, Loss: 2162.351, NoiseVar: 0.000, SignalVar: 1393.370, Lengthscale: 0.840, Epsilon: 0.268
Iter: 77, LR: 0.100, Loss: 2164.413, NoiseVar: 0.000, SignalVar: 1393.276, Lengthscale: 0.846, Epsilon: 0.272
Iter: 78, LR: 0.100, Loss: 2187.513, NoiseVar: 0.000, SignalVar: 1393.182, Lengthscale: 0.852, Epsilon: 0.275
Iter: 79, LR: 0.100, Loss: 2164.482, NoiseVar: 0.000, SignalVar: 1393.085, Lengthscale: 0.853, Epsilon: 0.276
Iter: 80, LR: 0.100, Loss: 2178.418, NoiseVar: 0.000, SignalVar: 1392.987, Lengthscale: 0.850, Epsilon: 0.275
Iter: 81, LR: 0.100, Loss: 2184.025, NoiseVar: 0.000, SignalVar: 1392.887, Lengthscale: 0.844, Epsilon: 0.273
Iter: 82, LR: 0.100, Loss: 2170.479, NoiseVar: 0.000, SignalVar: 1392.787, Lengthscale: 0.836, Epsilon: 0.270
Iter: 83, LR: 0.100, Loss: 2150.337, NoiseVar: 0.000, SignalVar: 1392.689, Lengthscale: 0.832, Epsilon: 0.269
Iter: 84, 

## Evaluation

In [11]:
%%capture
likelihood.eval()
model.eval()


## Metrics

In [12]:
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    preds_test = likelihood(model(test_x))

    mean_test = preds_test.mean
        
    error = test_y - preds_test.mean
    covar = preds_test.lazy_covariance_matrix.evaluate_kernel()
    inv_quad, logdet = covar.inv_quad_logdet(inv_quad_rhs=error.unsqueeze(-1), logdet=True)
    
    rmse = (error.square().sum()/test_y.shape[0]).sqrt()
    nll = 0.5 * sum([inv_quad, logdet, error.size(-1)* np.log(2 * np.pi)])/test_y.shape[0]
    
print("RMSE: ", rmse)
print("NLL: ", nll)

RMSE:  tensor(1.8487, device='cuda:0')
NLL:  tensor(12242703., device='cuda:0')


/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:324: NumericalWarning: CG terminated in 1000 iterations with average residual norm 1792697.375 which is larger than the tolerance of 1 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(
